In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [57]:
url_inicial = 'http://www.rmib.mx/index.php/rmib/issue/archive/6'
url_root = 'http://www.rmib.mx/index.php/rmib/issue/archive/6'
r=requests.get(url_inicial)

In [58]:
soup = BeautifulSoup(r.text, 'html.parser')

In [59]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://www.rmib.mx/index.php/rmib/issue/view/72',
 'http://www.rmib.mx/index.php/rmib/issue/view/59',
 'http://www.rmib.mx/index.php/rmib/issue/view/56',
 'http://www.rmib.mx/index.php/rmib/issue/view/68',
 'http://www.rmib.mx/index.php/rmib/issue/view/55',
 'http://www.rmib.mx/index.php/rmib/issue/view/66',
 'http://www.rmib.mx/index.php/rmib/issue/view/67',
 'http://www.rmib.mx/index.php/rmib/issue/view/54',
 'http://www.rmib.mx/index.php/rmib/issue/view/69']

In [60]:
def get_url_items(sopa, url): 
    box = soup.find('ul', class_='issues_archive')
    volumen=box.findAll('div', class_='obj_issue_summary')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [61]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina http://www.rmib.mx/index.php/rmib/issue/archive/6


In [62]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

9

In [63]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [64]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Ingeniería Biomédica'
    a='VII. Ingenierías'
    tem='Ciencias de la Salud'
    url='http://www.rmib.mx'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('div', class_='title').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [65]:
list_scraper=list_scraper[0:9]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8


In [66]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Revista Mexicana de Ingeniería Biomédica,VII. Ingenierías,Ciencias de la Salud,Detector de movimientos oculares,,http://www.rmib.mx,http://www.rmib.mx/index.php/rmib/article/view...
1,Revista Mexicana de Ingeniería Biomédica,VII. Ingenierías,Ciencias de la Salud,Sesión Biopotenciales,,http://www.rmib.mx,http://www.rmib.mx/index.php/rmib/article/view...
2,Revista Mexicana de Ingeniería Biomédica,VII. Ingenierías,Ciencias de la Salud,Seguridad eléctrica en el medio ambiente hospi...,,http://www.rmib.mx,http://www.rmib.mx/index.php/rmib/article/view...
3,Revista Mexicana de Ingeniería Biomédica,VII. Ingenierías,Ciencias de la Salud,Sesión de Informática médica,,http://www.rmib.mx,http://www.rmib.mx/index.php/rmib/article/view...
4,Revista Mexicana de Ingeniería Biomédica,VII. Ingenierías,Ciencias de la Salud,Calibración de equipo de radiodiagnóstico no f...,,http://www.rmib.mx,http://www.rmib.mx/index.php/rmib/article/view...
5,Revista Mexicana de Ingeniería Biomédica,VII. Ingenierías,Ciencias de la Salud,Tema ILos programas de enseñanza y su impacto ...,,http://www.rmib.mx,http://www.rmib.mx/index.php/rmib/article/view...
6,Revista Mexicana de Ingeniería Biomédica,VII. Ingenierías,Ciencias de la Salud,Salutación,,http://www.rmib.mx,http://www.rmib.mx/index.php/rmib/article/view...
7,Revista Mexicana de Ingeniería Biomédica,VII. Ingenierías,Ciencias de la Salud,Introducción al Procesamiento Discreto de Señales,,http://www.rmib.mx,http://www.rmib.mx/index.php/rmib/article/view...
8,Revista Mexicana de Ingeniería Biomédica,VII. Ingenierías,Ciencias de la Salud,Programa,,http://www.rmib.mx,http://www.rmib.mx/index.php/rmib/article/view...


In [67]:
df_catalogo.to_csv('Revista73.06.csv', index=False)